In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# --- 1. CONFIGURAZIONE ---
BASE_DIR = os.getcwd()
# Usiamo il file pulito al 100% (quello senza i 4 null)
INPUT_FILE = os.path.join(BASE_DIR, '../dataset', 'VAERS_READY_FOR_SPLIT.csv')

# Se per caso non trovi quel file, prova con quello del sanity check
if not os.path.exists(INPUT_FILE):
    INPUT_FILE = os.path.join(BASE_DIR, '../dataset', 'VAERS_SANITY_CHECKED.csv')

OUTPUT_TRAIN = os.path.join(BASE_DIR, '../dataset', 'train.csv')
OUTPUT_TEST = os.path.join(BASE_DIR, '../dataset', 'test.csv')

print(f"Caricamento dati da: {INPUT_FILE}")
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError("ERRORE: File di input non trovato.")

df = pd.read_csv(INPUT_FILE, low_memory=False)

# --- 2. CREAZIONE DELLA STRATIFICAZIONE AVANZATA ---
# Creiamo gruppi per garantire che il 20% di Test sia rappresentativo
# Età: 0-18, 19-64, 65+
bins = [-1, 18, 64, 150]
labels = ['Young', 'Adult', 'Senior']
df['AGE_BIN'] = pd.cut(df['AGE_YRS'], bins=bins, labels=labels)

# Colonna combinata: TARGET + SESSO + FASCIA ETÀ
# Es: "1_F_Senior"
df['STRATA'] = (
    df['IS_SEVERE'].astype(str) + "_" +
    df['SEX'] + "_" +
    df['AGE_BIN'].astype(str)
)

# Gestione casi rari (se un gruppo ha meno di 2 persone, non si può dividere)
# In quei casi rari, stratifichiamo solo su IS_SEVERE
counts = df['STRATA'].value_counts()
rare_groups = counts[counts < 2].index
df.loc[df['STRATA'].isin(rare_groups), 'STRATA'] = df.loc[df['STRATA'].isin(rare_groups), 'IS_SEVERE'].astype(str)

# --- 3. ESECUZIONE SPLIT 80/20 ---
print("\nEsecuzione Split (80% Train - 20% Test)...")

X = df.drop(columns=['IS_SEVERE'])
y = df['IS_SEVERE']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,      # <--- ECCO IL 20% PER IL TEST
    train_size=0.80,     # <--- ECCO L'80% PER IL TRAIN
    random_state=42,     # Garantisce che lo split sia sempre uguale
    stratify=df['STRATA'] # Mantiene le proporzioni demografiche
)

# --- 4. SALVATAGGIO ---
# Riuniamo X e y
train_df = X_train.copy()
train_df['IS_SEVERE'] = y_train

test_df = X_test.copy()
test_df['IS_SEVERE'] = y_test

# Rimuoviamo le colonne temporanee usate per il calcolo
cols_to_drop = ['STRATA', 'AGE_BIN']
train_df = train_df.drop(columns=cols_to_drop)
test_df = test_df.drop(columns=cols_to_drop)

train_df.to_csv(OUTPUT_TRAIN, index=False)
test_df.to_csv(OUTPUT_TEST, index=False)

# --- 5. REPORT FINALE ---
totale = len(df)
n_train = len(train_df)
n_test = len(test_df)

print("-" * 40)
print(f"✅ DATASET DIVISO CON SUCCESSO")
print(f"   Totale righe: {totale}")
print(f"   TRAIN Set (80%): {n_train} righe -> Salvato in 'data/train.csv'")
print(f"   TEST Set  (20%): {n_test} righe  -> Salvato in 'data/test.csv'")
print("-" * 40)

# Verifica % esatta
perc_train = (n_train / totale) * 100
perc_test = (n_test / totale) * 100
print(f"Verifica Percentuali Reali:")
print(f"Train: {perc_train:.2f}%")
print(f"Test:  {perc_test:.2f}%")

Caricamento dati da: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/raw/VAERS_READY_FOR_SPLIT.csv

Esecuzione Split (80% Train - 20% Test)...
----------------------------------------
✅ DATASET DIVISO CON SUCCESSO
   Totale righe: 838079
   TRAIN Set (80%): 670463 righe -> Salvato in 'data/train.csv'
   TEST Set  (20%): 167616 righe  -> Salvato in 'data/test.csv'
----------------------------------------
Verifica Percentuali Reali:
Train: 80.00%
Test:  20.00%


In [2]:
import pandas as pd
import os

# --- CONFIGURAZIONE ---
# Inserisci qui i percorsi corretti dei tuoi file
file_paths = {
    'Training Set': 'data/interim/splits/train_step6.csv',
    'Test Set': 'data/interim/splits/test_step6.csv',
    'Validation Set': 'data/interim/splits/val_step6.csv'
}

print(f"{'Dataset':<20} | {'Numero di Righe':<20}")
print("-" * 45)

# Dizionario per memorizzare i risultati (utile se vuoi farne un grafico dopo)
stats = {}

for label, path in file_paths.items():
    try:
        # Caricamento rapido: usiamo solo la prima colonna per contare le righe più velocemente
        # o shape[0] dopo il caricamento.
        # encoding='latin-1' è spesso necessario per i file VAERS.
        df = pd.read_csv(path, encoding='latin-1', low_memory=False)

        count = len(df)
        stats[label] = count

        # Stampa formattata con separatore delle migliaia
        print(f"{label:<20} | {count:>15,}")

    except FileNotFoundError:
        print(f"Errore: Il file '{path}' non è stato trovato.")
    except Exception as e:
        print(f"Errore durante la lettura di {label}: {e}")

print("-" * 45)

# Calcolo delle percentuali (utile per la tua slide sulla suddivisione 80/20)
total = sum(stats.values())
if total > 0:
    print("\nVerifica Suddivisione Percentuale:")
    for label, count in stats.items():
        percent = (count / total) * 100
        print(f"{label}: {percent:.2f}%")

Dataset              | Numero di Righe     
---------------------------------------------
Errore: Il file 'data/interim/splits/train_step6.csv' non è stato trovato.
Errore: Il file 'data/interim/splits/test_step6.csv' non è stato trovato.
Errore: Il file 'data/interim/splits/val_step6.csv' non è stato trovato.
---------------------------------------------
